<h1>Subspace Clustering</h1>

Subspace clustering attempts to find clusters in different subspaces of the dataset. Subspaces are projections of less dimensions than the full dataset, while they are part of it. For example, a line is a subspace of a plane.

First thigs first, subspaces. Suppose that _V_ is a vector space and _W_ is a subset of _V_, _W_ ⊆ _V_. Endow _W_ with the same operations as _V_. Then _W_ is a subspace if and only if three conditions are met (read more at [Beezer](http://linear.ups.edu/html/fcla.html)).
* _W_ is nonempty (_W_ ≠ ∅) and the zero vector belongs to _W_.
* If _x_∈_W_ and _y_∈_W_, then (_x_ + _y_)∈_W_.
* If α∈_C_ and _x_∈_W_, then α_x_∈_W_.

On the other hand, the clustering algorithms are different than regular clustering in full space, they vary not only on how they classify instances but how they tackle the subspaces. There are two main reviews on Subspace Clustering (SSC), *Subspace Clustering for High Dimensional Data: A Review* and *Evaluating Clustering in Subspace Projections of High Dimensional Data*. 

The first one divides the algorithms in two main groups, Bottom Up vs Top Down, in a very broad sense this means: 

* Bottom Up method starts by creating clusters in the lowest dimensions and adding those relevant. It takes advantage of the downward closure property of density, which means that if there are dense units (clusters) in k dimensions, there are dense units in all (k − 1) dimensional projections. The examples are: CLIQUE, ENCLUS, MAFIA, CBF, CLTree and DOC.

* Top Down finds clusters in the highest dimensional space, then iterates removing dimensions by their assigned weight. The exampels are: PROCLUS, ORCLUS, FINDIT and δ-Clusters.

The second one focuses on what they call 3 paradigms of clustering: cell-based, density-based and cluster focused.

* Cell-based: Sets of grid cells containg a certain amount of objects, such as CLIQUE, DOC, MINECLUS, and SCHISM 
* Density-based: Clusters are dense regions separated by sparse areas, such as SUBCLU, FIRES and INSCY
* Cluster-oriented: Determine properties of the set of clusers, like number, size or dimentionality. They present PROCLUS, P3C and STATPC.

There is an overlap in the following algorithms: PROCLUS (cluster-oriented/top-down), CLIQUE and DOC (grid-based/bottom-up). Further analysis on the algorithms will be done later, when required for implementation.

<h2>OpenSubspace</h2>

There is no Python implementation of SSC, and creating our own may be out of scope, especially since we are not sure it will work. So we will turn to an impementation in Weka (Waikato Environment for Knowledge Analysis, a machine learning suite) called  [OpenSubspace](http://dme.rwth-aachen.de/en/OpenSubspace) by the Data Management and Data Exploration Group of RWTH University. This implementation is based on the paper by Müller et al., *Evaluating Clustering in Subspace Projections of High Dimensional Data* so we focus on those available algorithms, especially those that overlap with the other review, adding FIRES because it is density based (and would be bottom-up) and the algorithm was provided by the original creators, while the rest are adapted for this implementation.

For this we will need to transform our data to Attribute-Relation File Format (`.arff`, the native Weka format) files, and run the algorithms in Java via command line.

In [ ]:
import arff

 The `arff.dump(object, file)` command takes a dictionary type object and encodes it into the required format. We have the 'Inverted Sentences' dictionary, but the arff structure is different, it looks like this:
 
<pre><code>@RELATION rev_vectors

@ATTRIBUTE dim_0 REAL
@ATTRIBUTE dim_1 REAL
@ATTRIBUTE 
@ATTRIBUTE dim_98 REAL
@ATTRIBUTE dim_99 REAL

@DATA
-7.75659233e-02, 4.02860008e-02, ..., 4.38096404e-01, -3.18264008e-01 (this for the first vector)
0.34120786, -0.23940383, ..., -0.45488459 -0.22274736
...
0.30884686, 0.4267047, ..., 0.84552544, -0.01002961
0.78674525 -0.14647771, ..., 0.07416188, -0.80752152 (this for the last vector, number 155847)
</pre></code>

Where the ATTRIBUTES will be the dimensions of our vectors, like columns in a relation, named something like dim_0, dim_1, ..., dim_99; and in DATA, it is all the values for the row in that column.

In dictionary type for Python, it should be like this:

<pre><code>
{u'attributes': [(u'dim_0', u'REAL'),
             (u'dim_1', u'REAL'),
             (u'...', u'REAL'),
             (u'dim_98', u'REAL'),
             (u'dim_99', u'REAL')],
 u'data': [[-7.75659233e-02, 4.02860008e-02, ..., 4.38096404e-01, -3.18264008e-01],
           [0.34120786, -0.23940383, ..., -0.45488459 -0.22274736],
           [...],
           [0.30884686, 0.4267047, ..., 0.84552544, -0.01002961],
           [0.78674525 -0.14647771, ..., 0.07416188, -0.80752152]],
 u'description': u'',
 u'relation': u'rev_vectors'}
 </pre></code>

In [ ]:
def vector_arff(labeled_sentences, model, name):
    """
    (LabeledSentences, doc2vec model, string) -> dict
    
    Requirements: 
    -All vectors are the same size
    
    Returns a dictionary ready for arff encoding.
    
    """
    
    vectors_arff = {}
    attributes = []
    data_arff = []
    
    for attribute in range(len(model.docvecs[sentences[0][1][0]])):
        attributes.append(tuple(('dim_{}'.format(str(attribute)), 'REAL')))
        
    for x in range(len(sentences)-1):
        vec = []
        vec = model.docvecs[sentences[x][1][0]]
        vec_list = vec.tolist()
        data_arff.append(vec_list)
    
    vectors_arff['relation'] = name
    vectors_arff['attributes'] = attributes
    vectors_arff['data'] = data_arff
    
    return vectors_arff
    print('Done')

Now we create the dictionary with the required arff structure using the desired corpus (in a list of LabeledSentence type) and the model, with a relation name. Then we encode it as an arff file.

In [ ]:
movie_arff = vector_arff(sentences, model, 'movies_amazon')

In [ ]:
arff.dump(movie_arff, open('movies.arff', 'w'))

movie_arff = arff.load(open('movies.arff', 'r'))

In [ ]:
def run_command(command):
    p = subprocess.Popen(command,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)
    return iter(p.stdout.readline, b'')

In [ ]:
output = []
for output_line in run_command("java -Xmx1024m -cp OpenSubspace\* weka.subspaceClusterer.Proclus -t OpenSubspace\data\Databases\synth_dbsizescale\S1500.arff"):
    output.append(output_line.decode())

The code to run the Subspace Clustering algorithm from here to the command line looks like this:

<code>subprocess.check_output(['java', '-Xmx:memory', '-cp', 'jar location', 'algorithm', -t', 'arff file', 'cluster args'])</code>

Some algorithm examples, with parameters and defaults:

* *PROCLUS* -K (4), -D (3): In PROCLUS, K and D are pretty self-explanatory, they mean the amount of clusters to find, as well as the average dimentionality of the clusters. It's a fairly quick algorithm, but it requires some knowledge or restriction of the clusters needed.

* *CLIQUE* -XI (10), -TAU (1.0): CLIQUE defines a cluster as a connection of grid cells with each more than τ (TAU) many objects. Grid cells are defined by a fixed grid splitting each dimension in ξ (XI) equal width cells.

* *MINECLUS* -a (0.08), -b (0.25), -w (0.2), -m (-1), -n (1), -k (5): MINECLUS is an optimization of DOC by using Frequent Pattern trees to improve decision time and accuracy, which is an optimization of CLIQUE itself, by using flexible hypercubes of width _W_ instead of a fixed grid. α (0,1] is the minimun density of the discovered clusters, β (0,1] is a balance condition between the number of points and number of dimensions in a cluster, the measure  M is the MAXOUT parameter, n is the number of bins and k ???

To work with several parameters, we can run experiments specifying the algorithm and tuning the values as necessary. It may not be intuitive and small changes can mean a large difference.

In [ ]:
k = [3]
d = [5,6]
proclus_experiments = {}
algorithm = 'Proclus'

from subprocess import Popen, PIPE, STDOUT
import time
for i in k:
    for j in d:
        start = time.time()
        results = []
        p = Popen(['java', '-Xmx2048m', '-cp', 'OpenSubspace\*', 'weka.subspaceClusterer.{0}'.format(algorithm),
                   '-t', 'pet_rev.arff', '-K', str(i), '-D', str(j) ], 
                  stdout=PIPE, stderr=STDOUT)
        for line in p.stdout:
            results.append(line.decode())
        proclus_experiments["proclus_S1500_k{0}_d{1}".format(i,j)] = results
        end = time.time()
        print('Done {0}_k{1}_d{2}'.format(algorithm,i,j), 'with clusters=', i, ', dimensions=', j, ' in ', (end-start)//60 , ' min')

In [ ]:
def Proclus_Experiment(clusters, dimensions, dictionary, dataset):
    algorithm = 'Proclus'    
    from subprocess import Popen, PIPE, STDOUT
    for i in clusters:
        for j in dimensions:
            start = time.time()
            results = []
            p = Popen(['java', '-Xmx4g', '-cp', 'OpenSubspace\*', 
                       'weka.subspaceClusterer.{0}'.format(algorithm), 
                       '-t', 'OpenSubspace\data\Databases\{}'.format(dataset), 
                       '-K', str(i), '-D', str(j) ], 
                       stdout=PIPE, stderr=STDOUT)
            for line in p.stdout:
                results.append(line.decode())
            dictionary["exp_k{0}_d{1}".format(i,j)] = results
            end = time.time()
            print('Done {0}_k{1}_d{2}'.format(algorithm,i,j), 'with clusters=', i, 
                  ', dimensions=', j, ' in ', (end-start), ' sec')
            
def Mineclus_Experiment(ALPHA, BETA, MAXOUT, K, numBins, W, dictionary, dataset):    
    mineclus_exp_pet = {}
    algorithm = 'MineClus'
    for i in ALPHA:
        for j in BETA:
            for k in MAXOUT:
                for l in K:
                    for m in numBins:
                        for n in W:
                            start = time.time()
                            results = []
                            p = Popen(['java', '-Xmx4g', '-cp',  'OpenSubspace\*', 
                                       'weka/subspaceClusterer/{0}'.format(algorithm),
                                       '-t', 'OpenSubspace\data\Databases\{}'.format(dataset), 
                                       '-a', str(i), '-b', str(j), '-m', str(k), 
                                       '-k', str(l), '-n', str(m), '-w', str(n)], 
                                      stdout=PIPE, stderr=STDOUT)
                            for line in p.stdout:
                                results.append(line.decode())
                            dictionary["exp_a{}_b{}_m{}_k{}_n{}_w{}".format(i,j,k,l,m,n)] = results
                            end = time.time()
                            print('Done {}_a{}_b{}_m{}_k{}_n{}_w{}'.format(algorithm,i,j,k,l,m,n), ' in ', (end-start), ' sec')

def clusters_detail(experiment):
    print(experiment[0], 'Number of Clusters', len(experiment)-3)
    for i in range(len(experiment)):
        s = experiment[i]
        if '[' in s:
            print('# instances in cluster {}:'.format(i-2), s[(s.index(']')+3):(s.index('{'))])
            print('dimensions of clusters:', s[(s.index('[')):(s.index(']')+1)])
    print('-'*100)
    
def clusters_info(experiment):
    print(experiment[0], 'Number of Clusters', len(experiment)-3)
    dims = []
    for i in range(len(experiment)):
        s = experiment[i]
        if '[' in s:
            #print('# instances in cluster {}:'.format(i-2), s[(s.index(']')+3):(s.index('{'))])
            dims.append(int((s[(s.index('[')):(s.index(']')+1)]).count('1')))
            #dims.append('1')
            #print(int((s[(s.index('[')):(s.index(']')+1)]).count('1')))
    if dims: print('Average dimensions =', str(round(np.mean(dims), 2)))   
    #print('dims=',dims)
    print('-'*100)

def sample_from_cluster(experiment, cluster, size):
    s = experiment[cluster+2]
    values = s[(s.index('{'))+1:(s.index('}')-1)].split(' ')
    sample = []
    if size > 0:
        for i in range(size): sample.append(int(random.choice(values)))
    else:
        for i in range(10): sample.append(int(random.choice(values)))
    return sample

def sample_sentences_from_cluster(experiment, cluster, size):
    s = experiment[cluster+2]
    values = s[(s.index('{'))+1:(s.index('}')-1)].split(' ')
    sample = []
    if size > 0:
        for i in range(size): 
            rdm = int(random.choice(values))
            sample.append(['Sentence #{}:'.format(i+1), sentences[rdm][1][0], ' '.join(sentences[rdm][0])])
    else:
        for i in range(10): sample.append('Sentence #{}:'.format(i+1) ,' '.join(sentences[int(random.choice(values))][0]))
    return sample

In [ ]:
k = [3]
d = [10, 20]
proclus_test = {}
Proclus_Experiment(k, d, proclus_test, 'movies.arff')

You are welcome to experiment with this, however we decided not to pursue the use of SSC because of the Extraction Method used in this work. The tuples resulting from doc2vec work best when all the dimensions are taken in account, so the effort on trying to get the most relevant dimensions is not so important. With some ideal algorithm that could name dimensions like "funinnes", "educational level", or "intensity of acting", then it may be interesting to look at the relevant dimensions for every movie.

In addition, it does not work well with cosine distances, we tried!